In [ ]:
import numpy as np
import glob
import os

# Relative path for input files
file_path = os.path.join('..', '..', 'dati_PJ', 'PJ_SG_*.dat')

# Initialize a list to accumulate data
all_data = []

# Initialize the expected number of lines
expected_rows = None

# Maximum number of files to consider
max_files = 100

# Count of read files
file_count = 0

# Read each file matching the pattern
for filename in glob.glob(file_path):
    if file_count >= max_files:
        break  # Stop reading if the limit of 100 files is reached
    with open(filename, 'r') as file:
        lines = file.readlines()
        values = []
        for line in lines[2:]:  # Skip the first two lines
            try:
                values.append(float(line.split()[1]))
            except ValueError:
                print(f"Conversion error in file {filename} at line: {line.strip()}")
                continue
        if expected_rows is None:
            expected_rows = len(values)
        elif len(values) != expected_rows:
            print(f"Error: file {filename} has a different number of lines ({len(values)}) compared to others ({expected_rows}).")
            continue  # Skip this file
        all_data.append(values)
        file_count += 1  # Increment the count of read files

# Check if there is data to process
if not all_data:
    print("No valid data found.")
else:
    # Convert the data to a numpy array for easier processing
    data_array = np.array(all_data)

    # Print the shape of the array for debugging
    print(f"Data array shape: {data_array.shape}")

    # Calculate the mean and standard deviation along the file axis (axis 0)
    means = np.mean(data_array, axis=0)
    std_devs = np.std(data_array, axis=0, ddof=1) / np.sqrt(data_array.shape[0])

    # Check the results of mean and standard deviation
    print(f"Number of elements in 'means': {len(means)}")
    print(f"Number of elements in 'std_devs': {len(std_devs)}")

    # Relative path for output file
    output_file_path = os.path.join('..', 'analisi_dati_PJ', 'PJ_media.dat')

    # Write the results to an output file
    with open(output_file_path, 'w') as output_file:
        output_file.write(f"# z: 3 h: 0.358 N: 10000000 Nfile: {file_count}\n")
        output_file.write("# 1: L 2:corr=0 3:error\n")
        for i in range(len(means)):
            output_file.write(f"{i+2} {means[i]} {std_devs[i]}\n")

print(f"Calculation finished using {file_count} files.")
